In [0]:
# from google.colab import drive
# drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import json
from pprint import pprint
import numpy as np
import re
import io
import nltk
nltk.download('punkt')
import json
import nltk
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Embedding, Dropout, Dense, Activation
from keras.layers import LSTM, Bidirectional, Input
from keras.layers import concatenate,merge
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, concatenate, Dropout, RepeatVector, Activation, merge, Lambda, Flatten, Reshape
from keras.layers import LSTM, Bidirectional, TimeDistributed, GRU, AveragePooling1D, Reshape, GlobalAveragePooling1D
from keras.layers import Recurrent, InputSpec
from keras.models import Model, load_model
from keras import initializers
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model
from keras import optimizers
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping


[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Using TensorFlow backend.


In [0]:
with open('train-v2.0.json') as data_file:
    data = json.load(data_file)

data_entity_list = []
for item in data['data']:
    for p in item['paragraphs']:
        for qq in p['qas']:
            for a in qq['answers']:
                data_entity_list.append((p['context'],qq['question'],a['text'],a['answer_start']))

with open('dev-v2.0.json') as data_file:
    data1 = json.load(data_file)

data_entity_list1 = []
for item in data1['data']:
    for p in item['paragraphs']:
        for qq in p['qas']:
            for a in qq['answers']:
                data_entity_list1.append((p['context'],qq['question'],a['text'],a['answer_start']))


In [0]:
import pandas as pd

df=pd.DataFrame(data_entity_list, columns=['context','question','answer','answer_start'])
val=pd.DataFrame(data_entity_list1, columns=['context','question','answer','answer_start'])

In [0]:
import re
df['answer_end']= 0
for i in range(len(df)):
    new=df['answer'][i].split()
    df['answer_end'][i]=(len(new)-1)+df['answer_start'][i]

val['answer_end']= 0
for i in range(len(val)):
    new=val['answer'][i].split()
    val['answer_end'][i]=(len(new)-1)+val['answer_start'][i]


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [0]:
X_train,y_train=df[['context','question']],df[['answer_start','answer_end']]
X_test,y_test=val[['context','question']],val[['answer_start','answer_end']]

In [0]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((86821, 2), (86821, 2), (20302, 2), (20302, 2))

In [0]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(X_train['context'])
vocab_size = len(tokenizer.word_index) + 1

In [0]:
vocab_size

86901

In [0]:
lenght_list=[]
for l in X_train.question:
    lenght_list.append(len(l.split(' ')))
max_ques= np.max(lenght_list)
max_ques

lenght_list=[]
for l in X_train.context:
    lenght_list.append(len(l.split(' ')))
max_doc= np.max(lenght_list)
max_doc


653

In [0]:
words = vocab_size
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = max_doc

f = open('glove.6B.100d.txt','r',encoding="utf8")
embedding_index = {}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs 
f.close()

embedding_matrix = np.zeros((words, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(words,EMBEDDING_DIM,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=True)

In [0]:
''' 
To perform vectorization of text records, 
this function is taken from 'https://github.com/wentaozhu/recurrent-attention-for-QA-SQUAD-based-on-keras' with necessary modifications.

Vectorize the words to their respective index and pad context to max context length and question to max question length.
Answers vectors are padded to the max context length as well.
'''
def vectorizeData(xContext, xQuestion, xAnswerBeing, xAnswerEnd, word_index, context_maxlen, question_maxlen):
    X = []
    Xq = []
    YBegin = []
    YEnd = []
    for i in range(len(xContext)):
        x = [word_index[w] if w in tokenizer.word_index else tokenizer.word_index['a']  for w in xContext[i]]
        xq = [word_index[w] if w in tokenizer.word_index else tokenizer.word_index['a'] for w in xQuestion[i] ]
        y_Begin =  np.zeros(len(xContext[i]))
        y_Begin[xAnswerBeing[i]] = 1
        y_End = np.zeros(len(xContext[i]))
        y_End[xAnswerEnd[i]] = 1
        X.append(x)
        Xq.append(xq)
        YBegin.append(y_Begin)
        YEnd.append(y_End)
    return pad_sequences(X, maxlen=context_maxlen, padding='post'), pad_sequences(Xq, maxlen=question_maxlen, padding='post'), pad_sequences(YBegin, maxlen=context_maxlen, padding='post'), pad_sequences(YEnd, maxlen=context_maxlen, padding='post')

In [0]:
X_train,y_train=X_train.reset_index() ,y_train.reset_index()
tX, tXq, tYBegin, tYEnd = vectorizeData(X_train['context'],X_train['question'],y_train['answer_start'],y_train['answer_end'],tokenizer.word_index,max_doc,max_ques)

In [0]:
''' 
To perform vectorization of text records, 
this function is taken from 'https://github.com/wentaozhu/recurrent-attention-for-QA-SQUAD-based-on-keras' with necessary modifications.

Vectorize the words to their respective index and pad context to max context length and question to max question length.
Answers vectors are padded to the max context length as well.
'''

def vectorizeValData(xContext, xQuestion, word_index, context_maxlen, question_maxlen):
    X = []
    Xq = []
    YBegin = []
    YEnd = []
    for i in range(len(xContext)):
        x = [word_index[w] if w in tokenizer.word_index else tokenizer.word_index['a']  for w in xContext[i]]
        xq = [word_index[w] if w in tokenizer.word_index else tokenizer.word_index['a'] for w in xQuestion[i] ]
        X.append(x)
        Xq.append(xq)

    return pad_sequences(X, maxlen=context_maxlen, padding='post'), pad_sequences(Xq, maxlen=question_maxlen, padding='post')

# vX, vXq = vectorizeValData(X_test['context'],X_test['question'], tokenizer.word_index,max_doc,max_ques)
vX, vXq, vStart, vEnd = vectorizeData(X_test['context'],X_test['question'],y_test['answer_start'],y_test['answer_end'],tokenizer.word_index,max_doc,max_ques)

In [0]:
'''
To calculate Bahdanau Attention, the given class is taken from 
https://github.com/wentaozhu/recurrent-attention-for-QA-SQUAD-based-on-keras/blob/master/rnnlayer.py , as it was already implemented and available.
'''
class Attention(Recurrent):
    def __init__(self, units, h_dim,
                 kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal',
                 **kwargs):
        self.units = units
        self.h_dim = h_dim
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        self.input_dim = input_shape[2] - self.h_dim

        if self.stateful:
            self.reset_states()
        else:
            self.states = [None]

        self.Wa = self.add_weight('{}_Wa'.format(self.name), (self.units, self.units),
                                  initializer=self.recurrent_initializer)
        self.Ua = self.add_weight('{}_Ua'.format(self.name), (self.h_dim, self.units),
                                  initializer=self.recurrent_initializer)
        self.Va = self.add_weight('{}_Va'.format(self.name),(self.units,1),
                                  initializer=self.recurrent_initializer)
        self.Wzr = self.add_weight('{}_Wzr'.format(self.name), (self.input_dim, 2 * self.units),
                                 initializer=self.recurrent_initializer)
        self.Uzr = self.add_weight('{}_Wzr'.format(self.name), (self.units, 2 * self.units),
                                 initializer=self.recurrent_initializer)
        self.Czr = self.add_weight('{}_Czr'.format(self.name), (self.h_dim, 2 * self.units),
                                   initializer=self.recurrent_initializer)
        self.W = self.add_weight('{}_W'.format(self.name), (self.input_dim, self.units),
                                 initializer=self.recurrent_initializer)
        self.U = self.add_weight('{}_U'.format(self.name), (self.units, self.units),
                                 initializer=self.recurrent_initializer)
        self.C = self.add_weight('{}_C'.format(self.name), (self.h_dim, self.units),
                                 initializer=self.recurrent_initializer)
        self.built = True

    def reset_states(self):
        assert self.stateful, 'Layer must be stateful.'
        input_shape = self.input_spec[0].shape
        if not input_shape[0]:
            raise ValueError('If a RNN is stateful, a complete '
                             'input_shape must be provided '
                             '(including batch size).')
        if hasattr(self, 'states'):
            K.set_value(self.states[0],
                        np.zeros((input_shape[0], self.units)))
        else:
            self.states = [K.zeros((input_shape[0], self.units))]

    def preprocess_input(self, inputs, training=None):
        return inputs

    def step(self, inputs, states):
        h_tm1 = states[0]  # previous memory
        h_tm1a = K.dot(h_tm1, self.Wa)
        eij = K.dot(K.tanh(h_tm1a + K.dot(inputs[:, :self.h_dim], self.Ua)), self.Va)
        eijs = K.repeat_elements(eij, self.h_dim, axis=1)

        cisum = eijs*inputs[:, :self.h_dim]
        zr = K.sigmoid(K.dot(inputs[:, self.h_dim:], self.Wzr) + K.dot(h_tm1, self.Uzr) + K.dot(cisum, self.Czr))
        zi = zr[:, :self.units]
        ri = zr[:, self.units: 2 * self.units]
        si_ = K.tanh(K.dot(inputs[:, self.h_dim:], self.W) + K.dot(ri*h_tm1, self.U) + K.dot(cisum, self.C))
        si = (1-zi) * h_tm1 + zi * si_
        return si, [si] #h_tm1, [h_tm1]

    def get_constants(self, inputs, training=None):
        constants = []
        constants.append([K.cast_to_floatx(1.) for _ in range(3)])
        return constants

    def get_config(self):
        config = {'units': self.units,
                  'kernel_initializer': initializers.serialize(self.kernel_initializer),
                  'recurrent_initializer': initializers.serialize(self.recurrent_initializer)}
        base_config = super(Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [0]:
question_input = Input(shape=(question_maxlen,), dtype='int32', name='question_input')
context_input = Input(shape=(context_maxlen,), dtype='int32', name='context_input')

questionEmbd = Embedding(output_dim=EMBEDDING_DIM, input_dim=vocab_size,
                         weights=[embedding_matrix], 
                         input_length=question_maxlen, trainable=False)(question_input)  
contextEmbd = Embedding(output_dim=EMBEDDING_DIM, input_dim=vocab_size,
                         weights=[embedding_matrix], 
                         input_length=context_maxlen, trainable=False)(context_input)  

In [0]:
Q_h = Bidirectional(LSTM(100, return_sequences=True))(questionEmbd)
C_h = Bidirectional(LSTM(100, return_sequences=True))(contextEmbd)

merge1 = concatenate([Q_h, C_h], axis=1)
atten_1 = Attention(100, 100, return_sequences=True)(merge1)
dropout_1 = Dropout(0.25)(atten_1)

QC_bilstm = Bidirectional(LSTM(100, return_sequences=True))(dropout_1)
merge2 = concatenate([QC_bilstm, C_h], axis=1)
dropout_2 = Dropout(0.5)(merge2)

start_bilstm =  Bidirectional(LSTM(100, return_sequences=True))(dropout_2)
end_bilstm =  Bidirectional(LSTM(100, return_sequences=True))(dropout_2)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
merge2.shape

TensorShape([Dimension(None), Dimension(None), Dimension(200)])

In [0]:
start_token_dense = LSTM(context_maxlen, activation='softmax')(start_bilstm)
end_token_dense = LSTM(context_maxlen, activation='softmax')(end_bilstm)


In [0]:
model = Model(input=[context_input, question_input], output=[start_token_dense, end_token_dense])
adam = optimizers.Adam(lr=0.003)
model.compile(optimizer=adam, loss='categorical_crossentropy')
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question_input (InputLayer)     (None, 60)           0                                            
__________________________________________________________________________________________________
context_input (InputLayer)      (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 60, 100)      10487700    question_input[0][0]             
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 300, 100)     10487700    context_input[0][0]              
__________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [0]:
tX = tX[:50000]
tXq = tXq[:50000]
tYBegin = tYBegin[:50000]
tYEnd = tYEnd[:50000]
vX = vX[:5000]
vXq = vXq[:5000]
vStart = vStart[:5000]
vEnd = vEnd[:5000]

tX.shape

(50000, 300)

In [0]:
checkpoint2 = ModelCheckpoint('squad_bilstm_with_atten1_v3.h5', monitor='val_lstm_6_loss', verbose=1, save_best_only=True, mode='min', period=2)
checkpoint3 = EarlyStopping(monitor='val_lstm_6_loss', min_delta=0.01, patience=7, verbose=1, mode='min')
checkpoint4 = EarlyStopping(monitor='val_lstm_7_loss', min_delta=0.01, patience=7, verbose=1, mode='min')
callbacks_list = [checkpoint2, checkpoint3, checkpoint4] 
EPOCHS = 10
model.fit([tX, tXq], [tYBegin,tYEnd], epochs=EPOCHS, batch_size=128, shuffle=True, 
          validation_split=0.2, \
          callbacks=callbacks_list, use_multiprocessing=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 3062s 77ms/step - loss: 10.6121 - lstm_6_loss: 5.2562 - lstm_7_loss: 5.3559 - val_loss: 10.4422 - val_lstm_6_loss: 5.1864 - val_lstm_7_loss: 5.2558
Epoch 2/10
40000/40000 [==============================] - 3046s 76ms/step - loss: 10.4012 - lstm_6_loss: 5.1836 - lstm_7_loss: 5.2177 - val_loss: 10.3816 - val_lstm_6_loss: 5.1688 - val_lstm_7_loss: 5.2128

Epoch 00002: val_lstm_6_loss improved from inf to 5.16880, saving model to squad_bilstm_with_atten1_v3.h5
Epoch 3/10
40000/40000 [==============================] - 3068s 77ms/step - loss: 10.3389 - lstm_6_loss: 5.1548 - lstm_7_loss: 5.1841 - val_loss: 10.3493 - val_lstm_6_loss: 5.1522 - val_lstm_7_loss: 5.1972
Epoch 4/10
40000/40000 [==============================] - 3092s 77ms/step - loss: 10.3178 - lstm_6_loss: 5.1341 - lstm_7_loss: 5.

In [0]:
# model.save('squad_bilstm_with_atten1_weights.h5')

In [0]:
# from keras.model import load_model
# trained_model = load_model('/content/squad_bilstm_with_atten1_weights.h5')

predictions = model.predict([vX, vXq], batch_size=128)
ansBegin = np.zeros((predictions[0].shape[0],), dtype=np.int32)
ansEnd = np.zeros((predictions[0].shape[0],),dtype=np.int32) 
for i in range(predictions[0].shape[0]):
	ansBegin[i] = predictions[0][i, :].argmax()
	ansEnd[i] = predictions[1][i, :].argmax()

In [2]:
# F1-Score calculation
from sklearn.metrics import f1_score

start_f1 = f1_score(vStart ,ansBegin, average="weighted")
end_f1 = f1_score(vEnd ,ansEnd, average="weighted")

#F1
(start_f1 + end_f1) / 2

0.578012

In [3]:
# Exact Match calculation

matched_token_count = 0

for t in range(len(ansBegin)):
  if ((ansBegin[t] == vStart[t]) and (ansEnd[t] == vEnd[t])):
    matched_token_count += 1

# EM
matched_token_count / len(ansBegin)

0.458